In [1]:
import numpy as np 
import tensorflow as tf 
import os
import time
from datetime import datetime
from sklearn.cross_validation import train_test_split

import data_helpers2

In [2]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape = shape, stddev = 0.1)
    return tf.Variable(initial)

def bias_variable(shape, constant=0.1):
    initial = tf.constant(value = constant, shape = shape)
    return tf.Variable(initial)


class TextCNN(object):

    def __init__(self, sequence_length, num_classes, vocab_size,
        embedding_size, filter_sizes, num_filters, l2_reg_lambda=0.0):

        # placeholders
        self.x = tf.placeholder(tf.int32, [None, sequence_length])
        self.y_ = tf.placeholder(tf.float32, [None, num_classes])
        self.keep_prob = tf.placeholder(tf.float32)
        # l2 regularization
        l2_loss = tf.constant(0.0)

        # embedding layer
        with tf.device('/cpu:0'), tf.name_scope('embedding'):
            W = tf.Variable(tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0))
            self.embedded_chars = tf.nn.embedding_lookup(W, self.x)
            self.embedded_chars_expanded = tf.expand_dims(self.embedded_chars, -1)
            
        # convolution layer
        pooled_outputs = []
        for i, filter_size in enumerate(filter_sizes):
            with tf.name_scope('conv-maxpool-%s' % filter_size):
                # conv
                filter_shape = [filter_size, embedding_size, 1, num_filters]
                W = weight_variable(filter_shape)
                b = bias_variable([num_filters])
                conv = tf.nn.conv2d(self.embedded_chars_expanded, W, strides = [1, 1, 1, 1], padding = 'VALID')
                hidden = tf.nn.relu(tf.nn.bias_add(conv, b))
                pooled = tf.nn.max_pool(hidden, ksize = [1, sequence_length - filter_size + 1, 1, 1],
                                       strides = [1, 1, 1, 1], padding = 'VALID')
                pooled_outputs.append(pooled)
                
        # combine all pooled
        num_filters_total = num_filters * len(filter_sizes)
        self.hidden_pool = tf.concat(3, pooled_outputs)
        self.hidden_pool_flat = tf.reshape(self.hidden_pool, [-1, num_filters_total])
        
        # dropout
        with tf.name_scope('dropout'):
            self.hidden_drop = tf.nn.dropout(self.hidden_pool_flat, self.keep_prob)
            
        # output layer
        with tf.name_scope('output'):
            W = weight_variable([num_filters_total, num_classes])
            b = bias_variable([num_classes])
            reg = tf.nn.l2_loss(W) + tf.nn.l2_loss(b)
            l2_loss += reg
            self.scores = tf.nn.xw_plus_b(self.hidden_drop, W, b)
            self.prediction = tf.argmax(self.scores, 1)
            
        # loss
        with tf.name_scope('loss'):
            losses = tf.nn.softmax_cross_entropy_with_logits(self.scores, self.y_)
            self.loss = tf.reduce_mean(losses) + l2_reg_lambda * l2_loss
            
        # evaluate
        with tf.name_scope('accuracy'):
            correct_pred = tf.equal(self.prediction, tf.argmax(self.y_, 1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
            

In [ ]:
embedding_dim = 128
filter_sizes = '3,4,5'
num_filters = 128
learning_rate = 1e-4
keep_prob = 0.5
l2_reg_lambda = 0
batch_size = 64
num_epochs = 200
allow_soft_placement = True
log_device_placement = False


# Get Data
X, y, vocabulary, vocabulary_inv = data_helpers2.load_data()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

print 'Vocabulary Size: {}'.format(len(vocabulary))
print 'Train / Test Split: {} / {}'.format(X_train.shape[0], X_test.shape[0])

# Lets Train

graph = tf.Graph()

with graph.as_default():
    session_config = tf.ConfigProto(
        allow_soft_placement = allow_soft_placement,
        log_device_placement = log_device_placement)
    
    sess = tf.Session(config = session_config)

    with sess.as_default():

        cnn = TextCNN(
            sequence_length = X_train.shape[1],
            num_classes = 2,
            vocab_size = len(vocabulary),
            embedding_size = embedding_dim,
            filter_sizes = map(int, filter_sizes.split(",")),
            num_filters = num_filters,
            l2_reg_lambda = l2_reg_lambda)
        
        global_step = tf.Variable(0, trainable = False)
        optimizer = tf.train.AdamOptimizer(learning_rate)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_optimizer = optimizer.apply_gradients(grads_and_vars, global_step = global_step)
        
        sess.run(tf.initialize_all_variables())
        
        
    def train_step(batch_x, batch_y):
        feed_dict = {cnn.x : batch_x, cnn.y_ : batch_y, cnn.keep_prob : keep_prob}
        _, step, loss, accuracy = sess.run([train_optimizer, global_step, cnn.loss, cnn.accuracy], 
                                           feed_dict = feed_dict)
        time_now = datetime.now().isoformat()
        if step % 50 == 0:
            print '{}: step {}, loss {:g}, acc {:g}'.format(time_now, step, loss, accuracy)
        
        
    def test_step(batch_x, batch_y):
        feed_dict = {cnn.x : batch_x, cnn.y_ : batch_y, cnn.keep_prob : 1.0}
        _, step, loss, accuracy = sess.run([train_optimizer, global_step, cnn.loss, cnn.accuracy], 
                                           feed_dict = feed_dict)
        time_now = datetime.now().isoformat()
        print '{}: step {}, loss {:g}, acc {:g}'.format(time_now, step, loss, accuracy)
        
    def run():
            batches = data_helpers2.batch_iter(zip(X_train, y_train), batch_size, num_epochs)
            
            for batch in batches:
                batch_x, batch_y = zip(*batch)
                train_step(batch_x, batch_y)
                current_step = tf.train.global_step(sess, global_step)
                if current_step % 100 == 0:
                    print '-----Test Accuracy-----'
                    test_step(X_test, y_test)
            print '----Training Finished-----'
            test_step(X_test, y_test)
                
                
if __name__ == '__main__':
    run()